In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from datetime import datetime, timedelta

In [4]:
#S = float(input('The actual price is:'))
S= 100
K = 110
T = 1
r = 0.05
q = 0
vol = 0.2

In [3]:
def F(S,r):
    F = S*np.exp(r)
    return F
print('The forward price is: {}$'.format(round(F(S,r),2)))

The forward price is: 105.13$


In [4]:
def Dr(r,T):
    Dr=np.exp(-r*T)
    return Dr

def Dq(q,T):
    Dq=np.exp(-q*T)
    return(Dq)

Dr=Dr(r,T)
Dq=Dq(q,T)
print(Dr,Dq)

0.951229424500714 1.0


In [7]:
def BSM(S,r,q,vol,T):
    Z=np.random.normal(0,1)
    St = S*np.exp((r-q-0.5*vol**2)*T+vol*np.sqrt(T)*Z)
    return St

St=BSM(S,r,q,vol,T)
print(St)

87.67717758174372


In [6]:
def d1_d2(S,K,r,q,vol,T):
    d1=(np.log(S/K)+(r-q+0.5*vol**2)*T)/(vol*np.sqrt(T))
    d2=d1-vol*np.sqrt(T)
    return d1, d2

d1, d2 = d1_d2(S,K,r,q,vol,T)
print(d1, d2)

if d2-d1-vol*np.sqrt(T)<1e-10:
    print('The formula is correct')
else:
    print('The formula seems incorrect')

-0.12655089902162442 -0.32655089902162443
The formula is correct


In [7]:
def call_put(S,Dq,K,Dr,d1,d2):
    N=norm.cdf
    call=S*Dq*N(d1)-K*Dr*N(d2)
    put=K*Dr*N(-d2)-S*Dq*N(-d1)
    return call,put

call,put = call_put(S,Dq,K,Dr,d1,d2)
print('The value of the call is {} and the value of the put is {} !'.format(round(call,2),round(put,2)))

if (call-put)-(S*Dq-K*Dr)<1e-10:
    print('The put call parity is respected')
else:
    print('The put call parity is violated')

The value of the call is 6.04 and the value of the put is 10.68 !
The put call parity is respected


In [9]:
def greeks (Dq,d1,S,vol,T,Dr,d2,r,K,q):
    phi=norm.pdf
    N=norm.cdf
    delta_c=Dq*N(d1)
    delta_p=Dq*(N(d1)-1)
    gamma=(Dq*phi(d1))/(S*vol*np.sqrt(T))
    vega=S*Dq*phi(d1)*np.sqrt(T)
    theta_c=-((S*Dq*phi(d1)*vol)/(2*np.sqrt(T)))-r*K*Dr*N(d2)+q*S*Dq*N(d1)
    theta_p=-((S*Dq*phi(d1)*vol)/(2*np.sqrt(T)))+r*K*Dr*N(-d2)-q*S*Dq*N(-d1)
    rho_c=K*T*Dr*N(d2)
    rho_p=-K*T*Dr*N(-d2)
    dividend_rho_c=-T*S*Dq*N(d1)
    dividend_rho_p=T*S*Dq*N(-d1)
    return delta_c,delta_p,gamma,vega,theta_c,theta_p,rho_c,rho_p,dividend_rho_c,dividend_rho_p

delta_c,delta_p,gamma,vega,theta_c,theta_p,rho_c,rho_p,dividend_rho_c,dividend_rho_p=greeks(Dq,d1,S,vol,T,Dr,d2,r,K,q)
print('The greeks value for delta_c,delta_p,gamma,vega,theta_c,theta_p,rho_c,rho_p,dividend_rho_c,dividend_rho_p are {}, {}, {}, {}, {}, {}, {}, {}, {}, {} !'.format(round(delta_c,2),round(delta_p,2),round(gamma,2),round(vega,2),round(theta_c,2),round(theta_p,2),round(rho_c,2),round(rho_p,2),round(dividend_rho_c,2),round(dividend_rho_p,2)))

The greeks value for delta_c,delta_p,gamma,vega,theta_c,theta_p,rho_c,rho_p,dividend_rho_c,dividend_rho_p are 0.45, -0.55, 0.02, 39.58, -5.9, -0.67, 38.92, -65.71, -44.96, 55.04 !


In [12]:
greeks_val = {
    "delta_c": delta_c,
    "delta_p": delta_p,
    "gamma": gamma,
    "vega": vega,
    "theta_c": theta_c,
    "theta_p": theta_p,
    "rho_c": rho_c,
    "rho_p": rho_p,
    "dividend_rho_c": dividend_rho_c,
    "dividend_rho_p": dividend_rho_p
}

In [11]:
def get_inputs():
    S = float(input('The underlying equal $'))
    K = float(input('The exercice price equal $'))
    T = float(input('The period equal $'))
    r = float(input('The riskfree rate equal $'))
    q = float(input('The yield equal $'))
    vol = float(input('The volatility equal $'))
    return S, K, T, r, q, vol

def Dr_Dq_F(S, K, T, r, q, vol):
    Dr = np.exp(-r*T)
    Dq = np.exp(-q*T)
    F  = S*np.exp(r)
    return Dr, Dq, F

def d1_d2(S, K, T, r, q, vol):
    d1 = (np.log(S/K) + (r-q+0.5*vol**2)*T) / (vol*np.sqrt(T))
    d2 = d1 - vol*np.sqrt(T)
    if d2-d1-vol*np.sqrt(T)<1e-10:
        print('The formulas for d2 and d1 are correct.')
    else:
        print('The formulas for d2 and d1 are incorrect.')  
    return d1, d2

def call_put(S, K, T, r, q, vol):
    Dr, Dq, _ = Dr_Dq_F(S, K, T, r, q, vol)
    d1, d2 = d1_d2(S, K, T, r, q, vol)
    N = norm.cdf
    call = S*Dq*N(d1) - K*Dr*N(d2)
    put  = K*Dr*N(-d2) - S*Dq*N(-d1)

    if (call-put)-(S*Dq-K*Dr)<1e-10:
        print('The put call parity is respected')
    else:
        print('The put call parity is violated')
    return call, put

def greeks(S, K, T, r, q, vol):
    Dr, Dq, F = Dr_Dq_F(S, K, T, r, q, vol)
    d1, d2 = d1_d2(S, K, T, r, q, vol)
    phi, N = norm.pdf, norm.cdf
    delta_c = Dq*N(d1)
    delta_p = Dq*(N(d1)-1)
    gamma   = (Dq*phi(d1))/(S*vol*np.sqrt(T))
    vega    = S*Dq*phi(d1)*np.sqrt(T)
    return delta_c, delta_p, gamma, vega

S, K, T, r, q, vol = get_inputs()
Dr,Dq,F = Dr_Dq_F(S, K, T, r, q, vol)
d1,d2 = d1_d2(S, K, T, r, q, vol)
call,put = call_put(S, K, T, r, q, vol)
delta_c,delta_p,gamma,vega=greeks(S, K, T, r, q, vol)
print('The value of the call is {} and the value of the put is {} !'.format(round(call,2),round(put,2)))
print('The greeks value for delta_c,delta_p,gamma,vega are {}, {}, {}, {} !'.format(round(delta_c,2),round(delta_p,2),round(gamma,2),round(vega,2)))

The underlying equal $ 100
The exercice price equal $ 110
The period equal $ 1
The riskfree rate equal $ 0.02
The yield equal $ 1
The volatility equal $ 0.2


The formulas for d2 and d1 are correct.
The formulas for d2 and d1 are correct.
The put call parity is respected
The formulas for d2 and d1 are correct.
The value of the call is 0.0 and the value of the put is 71.03 !
The greeks value for delta_c,delta_p,gamma,vega are 0.0, -0.37, 0.0, 0.0 !
